### Load libraries and external data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data import dataloader
import json
from neuralnet import nn
import numpy as np
import pixiedust_node

Pixiedust database opened successfully


pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.


#### Load data from Firebase.
Requires [Firebase service account credentials](https://console.firebase.google.com/project/tingle-pilot-collected-data/settings/serviceaccounts/adminsdk) in JSON format saved in `./firebase-credentials`.

In [3]:
pilot_data = dataloader.break_out_blocks(
    dataloader.load_from_firebase()
)

Data loaded from Firebase!


#### Load [Synaptic](http://caza.la/synaptic/)

In [8]:
%%node
var synaptic = require('synaptic');
var Neuron = synaptic.Neuron,
  Layer = synaptic.Layer,
  Network = synaptic.Network,
  Trainer = synaptic.Trainer,
  Architect = synaptic.Architect;

Define Perceptron and associated functions

In [9]:
%%node
/**
 * @constructor {function} Perceptron - Function to create synaptic perceptron
 * @see https://github.com/cazala/synaptic/blob/master/README.md
 * @param input
 * @param hidden
 * @param output
 * @this
 */
function Perceptron(input, hidden, output){
  // create the layers
  var inputLayer = new Layer(input);
  var hiddenLayer = new Layer(hidden);
  var outputLayer = new Layer(output);

  // connect the layers
  inputLayer.project(hiddenLayer);
  hiddenLayer.project(outputLayer);

  // set the layers
  this.set({
    input: inputLayer,
    hidden: [hiddenLayer],
    output: outputLayer
  });
}

// extend the prototype chain
Perceptron.prototype = new Network();
Perceptron.prototype.constructor = Perceptron;


/**
 * @function runPerceptronOnList - Function to run a perceptron on an
 * Array of Numeric input Arrays and return an Array of Numeric output Arrays.
 * @parameter {Perceptron} perceptron
 * @parameter {Numeric[][]} listOfInputs
 * @returns {Numeric[][]}
 */
function runPerceptronOnList(perceptron, listOfInputs){
  var outputs = [];
  for(var i=0; i<listOfInputs.length; i++){
    outputs.push(perceptron.activate(listOfInputs[i]));
  }
  return(outputs);
}

... ... ... ...
... ... ... ... ... ... ...
... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...
... ... ... ... ... ...
... ... ..... ..... ... ...


---
### See all targets, number of available samples and iteration blocks

In [4]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))

none: 19 on-target samples in 2 iteration blocks
environment: 106 on-target samples in 4 iteration blocks
body: 101 on-target samples in 2 iteration blocks
cup: 33 on-target samples in 3 iteration blocks
food: 35 on-target samples in 4 iteration blocks
nails: 69 on-target samples in 3 iteration blocks
smoke: 92 on-target samples in 3 iteration blocks
eyebrow: 235 on-target samples in 5 iteration blocks
nose: 249 on-target samples in 4 iteration blocks
above-ear: 403 on-target samples in 5 iteration blocks
behind-ear: 220 on-target samples in 4 iteration blocks
opposite-cheek: 74 on-target samples in 6 iteration blocks
chin: 166 on-target samples in 1 iteration block
cheek: 686 on-target samples in 6 iteration blocks
forehead: 176 on-target samples in 1 iteration block
top-head: 160 on-target samples in 1 iteration block
back-head: 170 on-target samples in 1 iteration block
opposite-face: 381 on-target samples in 2 iteration blocks
paint-mouth: 81 on-target samples in 3 iteration blocks

---
### Extract training and testing data
Define targets of interest and corresponding offtargets

In [5]:
with open(
    'data/targets.json',
    'r'
) as fp:
    targets = json.load(
        fp
    )[0]

Get training inputs and outputs, inputs that should evaluate ~true and inputs that should evaluate ~false

In [26]:
data = {}
for target in targets:
    data[target] = {
        "train": nn.define_trainer_data(
            pilot_data,
            {
                "target": [target],
                "offtarget": targets[target]
            },
            [
                "thermopile1",
                "thermopile2",
                "thermopile3",
                "thermopile4"
            ],
            list(range(1, 7)),
            100
        ),
        "test_true": nn.define_activation(
            pilot_data,
            [target],
            [
                "thermopile1",
                "thermopile2",
                "thermopile3",
                "thermopile4"
            ],
            list(range(1, 7)),
            exclude=100
        ),
        "test_false": nn.define_activation(
            pilot_data,
            targets[target],
            [
                "thermopile1",
                "thermopile2",
                "thermopile3",
                "thermopile4"
            ],
            list(range(1, 7)),
            exclude=100
        )
    }

Preview all inputs and training outputs

In [25]:
data

{'above-ear': {'test_false': [[79, 84.1, 84, 82.8],
   [79.3, 84.1, 83.8, 83],
   [80.1, 84, 84.8, 83],
   [78.9, 84.1, 84, 82.9],
   [82, 84.8, 81.5, 83.8],
   [79, 82.3, 79.5, 81.5],
   [81.9, 84, 80.8, 83.8],
   [82.4, 83.3, 80.6, 83.8],
   [84.1, 84, 80.6, 84.1],
   [80.1, 83.6, 81.6, 83.6],
   [81.4, 84, 81.4, 84],
   [81.6, 84, 80.8, 83.9],
   [79.8, 82.4, 80.8, 83.1],
   [81.9, 83.3, 81.3, 83.8],
   [82.6, 84, 81.9, 83.8],
   [82.6, 82.4, 81.1, 83.8],
   [79.3, 82.4, 81.8, 83],
   [80.6, 82.8, 81.1, 84.1],
   [82.1, 84, 81.1, 84],
   [79.8, 82.8, 81.1, 82.9],
   [79.8, 82.8, 81, 82.9],
   [80.4, 83.8, 81.3, 83.8],
   [82.3, 83.8, 81.4, 84],
   [80, 83, 81.4, 83.5],
   [80.5, 82.8, 81, 83.9],
   [80.5, 84, 82.1, 83.9],
   [82.3, 83.1, 81.3, 83.6],
   [80.8, 83.1, 80.6, 83.4],
   [80.3, 82.5, 80.6, 83.4],
   [81.5, 83.5, 81, 83.6],
   [81.3, 83.8, 81.3, 83.6],
   [81.3, 82.5, 80.8, 83.6],
   [80.5, 83.3, 81.3, 84],
   [81.9, 84, 81.3, 83.6],
   [80, 83, 80.9, 83.4],
   [80.6, 83.9

**Note**: These data take some time to copy across environments. Give the notebook some time between running cells across Python and JavaScript.

---
### Train and test

In [27]:
%%node
var test_outputs = {};
for(var target in data){
  var perceptron5 = new Perceptron(4,5,5,5,1);
  var perceptron7 = new Perceptron(4,7,7,7,7,7,1);
  var trainer5 = new Trainer(perceptron5);
  var trainer7 = new Trainer(perceptron7);
  test_outputs[target] = {
    "true": {
      "5": [],
      "7": []
    },
    "false": {
      "5": [],
      "7": []
    }
  };
  for(var iteration=0; iteration < data[target]["train"].length; iteration++){
    trainer5.train({
      "input": data[target]["train"][iteration]["input"],
      "output": data[target]["train"][iteration]["output"]
    });
    trainer7.train({
      "input": data[target]["train"][iteration]["input"],
      "output": data[target]["train"][iteration]["output"]
    });
  }
  for(var iteration=0; iteration < data[target]["test_true"].length; iteration++){
    test_outputs[target]["true"]["5"].push(perceptron5.activate(data[target]["test_true"][iteration]));
    test_outputs[target]["true"]["7"].push(perceptron7.activate(data[target]["test_true"][iteration]));
  }
  for(var iteration=0; iteration < data[target]["test_false"].length; iteration++){
    test_outputs[target]["false"]["5"].push(perceptron5.activate(data[target]["test_false"][iteration]));
    test_outputs[target]["false"]["7"].push(perceptron7.activate(data[target]["test_false"][iteration]));
  }
}

... ... ... ... ... ..... ....... ....... ....... ..... ....... ....... ....... ..... ... ..... ....... ....... ....... ..... ....... ....... ....... ..... ... ..... ..... ..... ... ..... ..... ..... ...


---
### See outputs

If the training is adequate x ≈ 0 ∀ x in the following:

In [28]:
f = [
    *[
        np.mean(output['5']) for output in [
            outputs['false'] for outputs in [
                test_outputs[i] for i in test_outputs
            ]
        ] if len(output['5'])
    ],
    *[
        np.mean(output['7']) for output in [
            outputs['false'] for outputs in [
                test_outputs[i] for i in test_outputs
            ]
        ] if len(output['7'])
    ]
]
f

[0.5072880935226826,
 0.5036263854810742,
 0.474161242787045,
 0.5136378378943367,
 0.49748453736387643,
 0.4873344581794078,
 0.4959980866859731,
 0.4983211994244817,
 0.5143300099939828,
 0.5076448862616922,
 0.5074797398203049,
 0.4880520545706865,
 0.48878986204466296,
 0.514339843218016,
 0.5111454267972476,
 0.49363435508129705,
 0.48777440206064165,
 0.5118734135561747,
 0.5004897727437816,
 0.5118053793028077,
 0.4982846368261045,
 0.4823123325974873,
 0.5026063686608289,
 0.516273322550322,
 0.5114410651663193,
 0.5018632710178133]

If the training is adequate x ≈ 1 ∀ x in the following:

In [29]:
t = [
    *[
        np.mean(output['5']) for output in [
            outputs['true'] for outputs in [
                test_outputs[i] for i in test_outputs
            ]
        ] if len(output['5'])
    ],
    *[
        np.mean(output['7']) for output in [
            outputs['true'] for outputs in [
                test_outputs[i] for i in test_outputs
            ]
        ] if len(output['7'])
    ]
]
t

[0.4741625606479557,
 0.5139048687143055,
 0.4975035579549104,
 0.4873332758251104,
 0.49831953493086567,
 0.514384009179432,
 0.5075997528874083,
 0.5074867025732086,
 0.48808188557025095,
 0.48879840281900866,
 0.4936420033727582,
 0.48769758362164356,
 0.5118736833720223,
 0.5004505388245333,
 0.4982680150431255,
 0.4823213699693807,
 0.5025362325050362,
 0.5165172996396232,
 0.5115187581595839,
 0.5019005977680056]

Finally, if training is adequate, f ≪ t:

In [31]:
f_mean = np.mean(f)
t_mean = np.mean(t)
print(
    "f = {0}\nt = {1}\n{0} ≪ {1} ?".format(
        str(f_mean),
        str(t_mean)
    ) if f_mean < t_mean else "Nope. f > t"
)

Nope. f > t
